A quick visualization script if I just want to see what the robot looks like in a given configuration.

# Setup

In [ ]:
## Imports

In [ ]:
# Numpy, Scipy, Matplotlib
import numpy as np

# Drake imports
import pydrake
from pydrake.all import (DiagramBuilder, RigidTransform, MathematicalProgram, RollPitchYaw,
                        RotationMatrix, Meshcat, MeshcatVisualizerParams, MeshcatVisualizerCpp,
                        InverseKinematics, Solve, SpatialInertia, UnitInertia)

# Imports of other project files
import constants
import config

import plant.simulation
import plant.manipulator as manipulator

import ctrl.aux
import plant.pedestal

import visualization

# Other imports
import time

## Drake

In [ ]:
meshcat = Meshcat()
web_url = meshcat.web_url()

## MBP

In [ ]:
builder = DiagramBuilder()

mbp_plant, scene_graph = pydrake.multibody.plant.AddMultibodyPlantSceneGraph(
    builder, time_step=1e-3)

In [ ]:
num_links = config.NumLinks.TWO
sys_consts = constants.nominal_sys_consts(num_links)

In [ ]:
# Arm
manipulator.data["add_plant_function"](
    mbp_plant,
    sys_consts.m_M,
    sys_consts.r,
    sys_consts.mu,
    scene_graph = scene_graph
)

In [ ]:
# Pedestal
pedestal_instance = plant.pedestal.AddPedestal(mbp_plant, num_links)

In [ ]:
# Paper
paper_instance = mbp_plant.AddModelInstance("paper")
paper_dims = [
    constants.PLYWOOD_LENGTH,
    sys_consts.w_L,
    sys_consts.h_L
]
paper_body = mbp_plant.AddRigidBody(
    "paper_body0", paper_instance,
    SpatialInertia(1, p_PScm_E=np.array([0., 0., 0.]),
                   G_SP_E=UnitInertia.SolidBox(*paper_dims))
)

mbp_plant.RegisterCollisionGeometry(
    paper_body, RigidTransform(), pydrake.geometry.Box(*paper_dims), "paper_body0",
    pydrake.multibody.plant.CoulombFriction(1,1)
)
mbp_plant.RegisterVisualGeometry(
    paper_body, RigidTransform(), pydrake.geometry.Box(*paper_dims),
    "paper_body0", [0, 1, 0, 1])

mbp_plant.WeldFrames(
    mbp_plant.GetFrameByName(plant.pedestal.pedestal_base_name, pedestal_instance),
    mbp_plant.GetBodyByName("paper_body0").body_frame(),
    RigidTransform(
        RotationMatrix().MakeZRotation(-np.pi/2),
        [0, 0, constants.PLYWOOD_LENGTH+sys_consts.h_L/2+plant.pedestal.PEDESTAL_BASE_Z_DIM/2])
)

In [ ]:
mbp_plant.Finalize()

## Diagram

In [ ]:
meshcat_params = MeshcatVisualizerParams()
vis = MeshcatVisualizerCpp.AddToBuilder(
    builder,
    scene_graph.get_query_output_port(),
    meshcat,
    meshcat_params)

diagram = builder.Build()
context = diagram.CreateDefaultContext()
mbp_context = mbp_plant.GetMyContextFromRoot(context)
vis_context = vis.GetMyContextFromRoot(context)

# Query

In [ ]:
traj_path = "/Users/dani/Documents/lis/notes/testing/2022-04-29/test1/qs/"
qs = np.load(traj_path + "qs.npz".format(30, 0.5), allow_pickle=True)["qs"]
q = qs[0]

In [ ]:
q

In [ ]:
dic = {}
for i, q_ in enumerate(q):
    dic["panda_joint" + str(i+1)] = q_
dic

# Visualize

In [ ]:
mbp_plant.SetPositions(mbp_context, q)
vis.Publish(vis_context)